In [1]:
import os
import re
import pandas as pd
import yaml

import wombat.bsub as bsub

In [2]:
# !pip install git+https://github.com/estorrs/wombat.git

In [3]:
# !pip uninstall wombat -y

###### grab data from cptac bammap

In [4]:
df = pd.read_csv('/home/estorrs/CPTAC3.catalog/BamMap/storage1.BamMap.dat', sep='\t', index_col=0)
df = df[df['experimental_strategy']=='WXS']
df = df[df['sample_type']=='blood_normal']
df = df[df['reference']=='hg38']
df

,case,disease,experimental_strategy,sample_type,data_path,filesize,data_format,reference,UUID,system
# sample_name,,,,,,,,,,
C3L-00081.WXS.N.hg38,C3L-00081,LSCC,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,39312702311,BAM,hg38,ec948c00-910b-4c7b-82a7-4d209d377116,storage1
C3L-00137.WXS.N.hg38,C3L-00137,UCEC,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,29560881189,BAM,hg38,d5a11391-8b84-4be2-8a8e-62d1e646b786,storage1
C3L-00938.WXS.N.hg38,C3L-00938,UCEC,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,41929157127,BAM,hg38,98c830a5-8735-4f5f-812d-3e7f7c3aa514,storage1
C3L-01039.WXS.N.BIOTEXT_kRQkDBr.hg38,C3L-01039,PDA,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,36998045601,BAM,hg38,3d9d35d6-5d4c-4585-bdd9-7fd46867519f,storage1
C3L-01232.WXS.N.hg38,C3L-01232,GBM,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,45532138913,BAM,hg38,a65f1db8-170a-4ec2-a20b-e6deff964e2c,storage1
...,...,...,...,...,...,...,...,...,...,...
C3N-04693.WXS.N.hg38,C3N-04693,GBM,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,40597818463,BAM,hg38,6fb6bf47-d501-4858-ba49-6e2cd516474a,storage1
C3N-04695.WXS.N.hg38,C3N-04695,GBM,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,32304960170,BAM,hg38,3bbdc094-e064-4c89-afec-bf59940cfe16,storage1
C3N-05039.WXS.N.hg38,C3N-05039,GBM,WXS,blood_normal,/storage1/fs1/m.wyczalkowski/Active/Primary/CP...,41040654784,BAM,hg38,b528bf8e-bae9-4d91-8d62-1746ba1a1ac0,storage1


calls from https://github.com/ding-lab/GATK4SCNA

In [5]:
# GENOME=/storage1/fs1/dinglab/Active/Projects/austins2/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa
# GENOME_DICT=/storage1/fs1/dinglab/Active/Projects/austins2/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict


# # Target
# TargetIntervalList=/code/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list


# # SNP database
# COMMON_BIALLELIC=/code/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf


# # protein coding genes
# ProteinCodingGene=/code/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed


In [6]:
genome = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'
genome_dict = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'
target_interval_list = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list'
common_biallelic = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
protein_coding_gene = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed'







In [7]:
pon_out_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon'

In [20]:
args = bsub.DEFAULT_ARGS
args['n_concurrent'] = 25
args['mem'], args['max_mem'] = 28, 28
args['docker'] = 'estorrs/pecgs_cnv:0.0.1'
args['group_name'] = 'pecgs-25'
args

{'mem': 28,
 'max_mem': 28,
 'docker': 'estorrs/pecgs_cnv:0.0.1',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'pecgs-25',
 'n_concurrent': 25,
 'interactive': False,
 'username': 'estorrs'}

###### step 1: precall

In [21]:
# bash /storage1/fs1/dinglab/Active/Projects/austins2/tools/GATK4SCNA/gatk_somatic.cnv.mgi.sh -p precall -t /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/CNV/gatk4scna/bam.list -M /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/CNV/gatk4scna/normalBam.list -o /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/CNV/gatk4scna -c /storage1/fs1/dinglab/Active/Projects/austins2/tools/GATK4SCNA/config/config.gatk4scna.mgi.ini

# sh $submitJob 4 1 gatk4cn.s1.${name} "bash $scriptDir/src/1.run.gatk4scna.collectReadCounts.Normal.sh -C ${config} -N ${caseID} -T Normal -B ${normalBam} -O ${outdir}"




In [22]:
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx4g -jar ${GATK} CollectReadCounts \
#       -I ${BAM} \
#       -L ${TargetIntervalList} \
#       --interval-merging-rule OVERLAPPING_ONLY \
#       -O ${OUT}/${NAME}.N.counts.hdf5

In [23]:
def get_precall_command(sample, normal_fp, out_dir):
    pieces = [
        'gatk CollectReadCounts',
        '-I', normal_fp,
        '-L', target_interval_list,
        '-interval-merging-rule OVERLAPPING_ONLY',
        f'-O {out_dir}/{sample}.N.counts.hdf5'
    ]
    cmd = ' '.join(pieces)
    return cmd


In [24]:
precall_out_dir = os.path.join(pon_out_dir, 'precall')
cmds = []
for i, row in df.iterrows():
    cmd = f'export PATH=/miniconda/envs/cnv/bin:$PATH && ' + get_precall_command(row['case'], row['data_path'], precall_out_dir)
    cmds.append(cmd)
cmds[:2]

['export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CollectReadCounts -I /storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/ec948c00-910b-4c7b-82a7-4d209d377116/5e04faec-58e8-403f-942b-74e8c0053805_gdc_realn.bam -L /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list -interval-merging-rule OVERLAPPING_ONLY -O /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00081.N.counts.hdf5',
 'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CollectReadCounts -I /storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/d5a11391-8b84-4be2-8a8e-62d1e646b786/552d7534-1fd1-490b-8751-9dff1bf55569_gdc_realn.bam -L /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list -interval-merging-rule 

In [25]:
job_names = [f'{s}_precall' for s in df['case']]
volumes = ['/storage1/fs1/m.wyczalkowski/Active', '/storage1/fs1/dinglab/Active']
bsub_commands = bsub.batch_bsub_commands(cmds, job_names, os.path.join(precall_out_dir, 'logs'), args, volumes=volumes)
bsub_commands[:3]

Failed to create log dir /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/logs


['export LSF_DOCKER_VOLUMES="/storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/logs:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/logs"',
 'bgadd -L 25 /estorrs/pecgs-25',
 "bsub -R 'rusage[mem=28GB]' -M 28GB -q dinglab -G compute-dinglab -a 'docker(estorrs/pecgs_cnv:0.0.1)' -g /estorrs/pecgs-25 -J C3L-00081_precall -oo /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/logs/C3L-00081_precall.txt 'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CollectReadCounts -I /storage1/fs1/m.wyczalkowski/Active/Primary/CPTAC3.share/CPTAC3-GDC/GDC_import/data/ec948c00-910b-4c7b-82a7-4d209d377116/5e04faec-58e8-403f-942b-74e8c0053805_gdc_realn.bam -L /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.pr

In [26]:
bsub.write_command_file(bsub_commands, '../tests/1.precall.sh')

In [27]:
bsub.write_command_file(bsub_commands[:3], '../tests/1.precall_small.sh')

###### step 2: pool normal

In [28]:
# sh $submitJob 8 1 gatk4cn.s2.pon "bash $scriptDir/src/2.run.gatk4scna.createPON.sh -C ${config} -D ${hdf5Dir}"

In [ ]:
# all_normal_hdf5=`ls ${DIR}/*.N.counts.hdf5 | while read file; do printf -- "-I $file "; done`
# ${JAVA} -Dsamjdk.use_async_io_read_samtools=false \
#       -Dsamjdk.use_async_io_write_samtools=true \
#       -Dsamjdk.use_async_io_write_tribble=false \
#       -Dsamjdk.compression_level=2 \
#       -Xmx8g -jar ${GATK} CreateReadCountPanelOfNormals \
#       --minimum-interval-median-percentile 5.0 \
#       -O ${DIR}/gatk4scnaPON.Normal.hdf5 \
#       $all_normal_hdf5

In [32]:
def get_pool_command(sample, fps, out_dir):
    pieces = [
        'gatk CreateReadCountPanelOfNormals',
        '--minimum-interval-median-percentile 5.0',
        f'-O {out_dir}/gatk4scnaPON.Normal.hdf5'
    ]
    pieces += [f'-I {fp}' for fp in fps]
    
    cmd = ' '.join(pieces)
    return cmd


In [33]:
pool_out_dir = os.path.join(pon_out_dir, 'create_pon')

In [34]:
fps = [f'{precall_out_dir}/{s}.N.counts.hdf5' for s in df['case'].to_list()]
fps

['/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00081.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00137.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00938.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-01039.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-01232.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-02204.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-02644.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-02662.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-02888.N.counts.hdf5',
 '/storage1/fs1/dinglab/Active/Projects/estorr

In [36]:
cmd = f'export PATH=/miniconda/envs/cnv/bin:$PATH && ' + get_pool_command(df['case'], fps, pool_out_dir)
cmd

'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CreateReadCountPanelOfNormals --minimum-interval-median-percentile 5.0 -O /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00081.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00137.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00938.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-01039.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-01232.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-02204.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-02644.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Pr

In [38]:
job_names = [f'create_pon']
volumes = ['/storage1/fs1/m.wyczalkowski/Active', '/storage1/fs1/dinglab/Active']
bsub_commands = bsub.batch_bsub_commands([cmd], ['create_pon'], pool_out_dir, args, volumes=volumes)
bsub_commands

Failed to create log dir /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon


['export LSF_DOCKER_VOLUMES="/storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon"',
 'bgadd -L 25 /estorrs/pecgs-25',
 "bsub -R 'rusage[mem=28GB]' -M 28GB -q dinglab -G compute-dinglab -a 'docker(estorrs/pecgs_cnv:0.0.1)' -g /estorrs/pecgs-25 -J create_pon -oo /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon/create_pon.txt 'export PATH=/miniconda/envs/cnv/bin:$PATH && gatk CreateReadCountPanelOfNormals --minimum-interval-median-percentile 5.0 -O /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/precall/C3L-00081.N.counts.hdf5 -I /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cn

In [ ]:
bsub.write_command_file(bsub_commands, '../tests/1.precall.sh')